In [2]:
import logging
import time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import os
import xml.etree.ElementTree as Xet
import re
import matplotlib.pyplot as plt
from platform import python_version
from sklearn.metrics import roc_auc_score
from torch.autograd import Variable

plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [53]:
my_df = pd.read_csv('C:\\Users\\przem\\Downloads\\data.csv')
my_df=my_df.drop(['gender', 'extroverted','age_group','Unnamed: 0','id','stable','conscientious','agreeable'], axis=1)
my_df['document'] = my_df['document'].apply(lambda x : x.lower())
my_df['document'] = my_df['document'].apply(lambda x : re.sub("@[A-Za-z0-9_]+","", x))
my_df['document'] = my_df['document'].apply(lambda x : re.sub("#[A-Za-z0-9_]+","", x))
my_df['document'] = my_df['document'].apply(lambda x : re.sub("http\S+","", x))
my_df['document'] = my_df['document'].apply(lambda x : re.sub("[()!?]"," ", x))
my_df['document'] = my_df['document'].apply(lambda x : re.sub("\[.*?\]"," ", x))
my_df['document'] = my_df['document'].apply(lambda x : re.sub("[^a-z]"," ", x))
target_feature=my_df['openx']
my_df['target']=(target_feature-min(target_feature))/(max(target_feature)-min(target_feature))
my_df=my_df.drop(['openx'], axis=1)
my_df.rename({'document': 'text'}, axis=1, inplace=True)

In [54]:
my_df.mean()

target    0.576309
dtype: float64

In [55]:
my_df

,text,target
0,things i want for my business cards but are to...,1.000000
1,painters produced their most highly valued wo...,1.000000
2,your new discussion layout is confusing regar...,1.000000
3,i never really understood why game environment...,1.000000
4,k and on a gun fine but throwing th...,1.000000
...,...,...
14161,fifty writing tools quick list poynter,0.333333
14162,video how to make vietnamese coffee by highb...,0.333333
14163,lyx is soooo awesome finally figured out ho...,0.333333
14164,impact algorithms strategies remarkable peopl...,0.333333


In [7]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    14166 non-null  object 
 1   target  14166 non-null  float64
dtypes: float64(1), object(1)
memory usage: 221.5+ KB


In [8]:
x = my_df.text
y = my_df.target

In [9]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [10]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [11]:
def labelize_tweets(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [12]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets(all_x, 'all')

In [13]:
cores = multiprocessing.cpu_count()
model_cbow = Word2Vec(sg=0, vector_size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|███████████████████████████████████████████████████████████████████████| 14166/14166 [00:00<00:00, 2020832.27it/s]


In [14]:
%%time
for epoch in range(30):
    model_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_cbow.alpha -= 0.002
    model_cbow.min_alpha = model_cbow.alpha

100%|███████████████████████████████████████████████████████████████████████| 14166/14166 [00:00<00:00, 2355834.84it/s]


Wall time: 2.71 s


In [15]:
model_sg = Word2Vec(sg=1, vector_size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|███████████████████████████████████████████████████████████████████████| 14166/14166 [00:00<00:00, 2365495.28it/s]


In [16]:
%%time
for epoch in range(30):
    model_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_sg.alpha -= 0.002
    model_sg.min_alpha = model_sg.alpha

100%|███████████████████████████████████████████████████████████████████████| 14166/14166 [00:00<00:00, 2362673.39it/s]


Wall time: 3.31 s


In [17]:
model_cbow.save('w2v_model_cbow.word2vec')
model_sg.save('w2v_model_sg.word2vec')

In [18]:
from gensim.models import KeyedVectors
model_cbow = KeyedVectors.load('w2v_model_cbow.word2vec')
model_sg = KeyedVectors.load('w2v_model_sg.word2vec')

In [19]:
len(model_cbow.wv.key_to_index.keys())

7808

In [ ]:
embeddings_index = {}
for w in model_cbow.wv.key_to_index.keys():
    embeddings_index[w] = np.append(model_cbow.wv[w],model_sg.wv[w])

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=7808)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [22]:
len(tokenizer.word_index)

18100

In [24]:
sequences[:3]

[[81, 30, 2, 1519, 5114, 12, 111, 306, 2039, 2635, 12, 245, 68, 63, 2, 1053],
 [327, 2040, 8, 5115, 5116, 38, 3098],
 [2636, 16, 11, 411, 2041, 362]]

In [25]:
length = []
for x in x_train:
    length.append(len(x.split()))

In [26]:
max(length)

33

In [ ]:
maxlen=35

In [27]:
x_train_seq = pad_sequences(sequences, maxlen=maxlen)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (13882, 35)


In [28]:
x_train_seq[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,   81,   30,    2,
        1519, 5114,   12,  111,  306, 2039, 2635,   12,  245,   68,   63,
           2, 1053],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  327, 2040,    8, 5115, 5116,
          38, 3098],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0, 2636,   16,   11,  411,
        2041,  362],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 1189, 2042, 3879,
        7718,  195],
       [   0,    0,    0,   

In [29]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=maxlen)

In [30]:
num_words = 18100
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [32]:
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [34]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=maxlen)

In [35]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [36]:
model_cnn_01 = Sequential()
e = Embedding(num_words, 200, weights=[embedding_matrix], input_length=maxlen, trainable=False)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(1, activation='sigmoid'))
model_cnn_01.compile(loss='mse', optimizer='adam')
model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
434/434 - 3s - loss: 0.0578 - val_loss: 0.0496 - 3s/epoch - 6ms/step
Epoch 2/5
434/434 - 2s - loss: 0.0496 - val_loss: 0.0502 - 2s/epoch - 5ms/step
Epoch 3/5
434/434 - 2s - loss: 0.0427 - val_loss: 0.0526 - 2s/epoch - 5ms/step
Epoch 4/5
434/434 - 2s - loss: 0.0368 - val_loss: 0.0603 - 2s/epoch - 5ms/step
Epoch 5/5
434/434 - 2s - loss: 0.0319 - val_loss: 0.0548 - 2s/epoch - 5ms/step


In [37]:
model_cnn_02 = Sequential()
e = Embedding(num_words, 200, input_length=maxlen)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
model_cnn_02.compile(loss='mse', optimizer='adam')
model_cnn_02.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
434/434 - 21s - loss: 0.0541 - val_loss: 0.0493 - 21s/epoch - 48ms/step
Epoch 2/5
434/434 - 20s - loss: 0.0337 - val_loss: 0.0563 - 20s/epoch - 47ms/step
Epoch 3/5
434/434 - 20s - loss: 0.0174 - val_loss: 0.0522 - 20s/epoch - 47ms/step
Epoch 4/5
434/434 - 20s - loss: 0.0115 - val_loss: 0.0490 - 20s/epoch - 47ms/step
Epoch 5/5
434/434 - 21s - loss: 0.0084 - val_loss: 0.0543 - 21s/epoch - 48ms/step


In [38]:
model_cnn_03 = Sequential()
e = Embedding(num_words, 200, weights=[embedding_matrix], input_length=maxlen, trainable=True)
model_cnn_03.add(e)
model_cnn_03.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_03.add(GlobalMaxPooling1D())
model_cnn_03.add(Dense(256, activation='relu'))
model_cnn_03.add(Dense(1, activation='sigmoid'))
model_cnn_03.compile(loss='mse', optimizer='adam')
model_cnn_03.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
434/434 - 22s - loss: 0.0575 - val_loss: 0.0489 - 22s/epoch - 51ms/step
Epoch 2/5
434/434 - 21s - loss: 0.0450 - val_loss: 0.0463 - 21s/epoch - 47ms/step
Epoch 3/5
434/434 - 20s - loss: 0.0336 - val_loss: 0.0481 - 20s/epoch - 47ms/step
Epoch 4/5
434/434 - 20s - loss: 0.0216 - val_loss: 0.0504 - 20s/epoch - 47ms/step
Epoch 5/5
434/434 - 20s - loss: 0.0144 - val_loss: 0.0506 - 20s/epoch - 46ms/step


In [39]:
from keras.layers.recurrent import LSTM
from keras.layers import Activation, TimeDistributed

lstm1 = Sequential()
e=Embedding(num_words, 200, weights=[embedding_matrix], input_length=maxlen, trainable=False)
lstm1.add(e)
lstm1.add(LSTM(units=200))
lstm1.add(Dense(2))
lstm1.add(Activation('softmax'))
lstm1.compile(loss='mse', optimizer='adam')
lstm1.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
434/434 - 36s - loss: 0.0664 - val_loss: 0.0582 - 36s/epoch - 83ms/step
Epoch 2/5
434/434 - 31s - loss: 0.0662 - val_loss: 0.0581 - 31s/epoch - 72ms/step
Epoch 3/5
434/434 - 31s - loss: 0.0662 - val_loss: 0.0581 - 31s/epoch - 72ms/step
Epoch 4/5
434/434 - 31s - loss: 0.0662 - val_loss: 0.0581 - 31s/epoch - 71ms/step
Epoch 5/5
434/434 - 33s - loss: 0.0662 - val_loss: 0.0581 - 33s/epoch - 75ms/step


In [40]:
lstm2 = Sequential()
e = Embedding(num_words, 200, input_length=maxlen)
lstm2.add(e)
lstm2.add(LSTM(units=200))
lstm2.add(Dense(2))
lstm2.add(Activation('softmax'))
lstm2.compile(loss='mse', optimizer='adam')
lstm2.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
434/434 - 62s - loss: 0.0662 - val_loss: 0.0581 - 62s/epoch - 144ms/step
Epoch 2/5
434/434 - 59s - loss: 0.0662 - val_loss: 0.0581 - 59s/epoch - 137ms/step
Epoch 3/5
434/434 - 59s - loss: 0.0662 - val_loss: 0.0581 - 59s/epoch - 136ms/step
Epoch 4/5
434/434 - 58s - loss: 0.0662 - val_loss: 0.0581 - 58s/epoch - 134ms/step
Epoch 5/5
434/434 - 58s - loss: 0.0662 - val_loss: 0.0581 - 58s/epoch - 134ms/step


In [41]:
lstm3 = Sequential()
e = Embedding(num_words, 200, weights=[embedding_matrix], input_length=maxlen, trainable=True)
lstm3.add(e)
lstm3.add(LSTM(units=200))
lstm3.add(Dense(2))
lstm3.add(Activation('softmax'))
lstm3.compile(loss='mse', optimizer='adam')
lstm3.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

Epoch 1/5
434/434 - 62s - loss: 0.0664 - val_loss: 0.0581 - 62s/epoch - 143ms/step
Epoch 2/5
434/434 - 60s - loss: 0.0662 - val_loss: 0.0581 - 60s/epoch - 138ms/step
Epoch 3/5
434/434 - 54s - loss: 0.0662 - val_loss: 0.0581 - 54s/epoch - 123ms/step
Epoch 4/5
434/434 - 49s - loss: 0.0662 - val_loss: 0.0581 - 49s/epoch - 112ms/step
Epoch 5/5
434/434 - 48s - loss: 0.0662 - val_loss: 0.0581 - 48s/epoch - 111ms/step


In [48]:
from sklearn.metrics import mean_squared_error
results=[]
models=[model_cnn_01,model_cnn_02,model_cnn_03,lstm1,lstm2,lstm3]
for m in models:
    m.evaluate(x=x_test_seq, y=y_test)
    sequences_test = tokenizer.texts_to_sequences(x_test)
    x_test_seq = pad_sequences(sequences_test, maxlen=maxlen)
    y_pred=m.predict(x_test_seq)
    corr_y_pred=[]
    for i in y_pred:
        corr_y_pred.append(i[0])
    rms = mean_squared_error(y_test, corr_y_pred, squared=False)
    results.append(rms)

5/5 [==============================] - 0s 18ms/step - loss: 0.0763


In [49]:
results

[0.24325999158246783,
 0.23574411228558845,
 0.24901063811607702,
 0.2758464476557353,
 0.27620934328313484,
 0.27649455033626213]

In [50]:
"""
cnn1: word2vec embdedding + not trainable during fitting
cnn2: default embdedding
cnn2: word2vec embdedding + trainable during fitting

REGRESSION:

EXTROVERTED

cnn1:  0.20138032381716772
cnn2:  0.19798711781557132
cnn3:  0.1940063193601567
lstm1: 0.23499138675457792
lstm2: 0.23476451093520545
lstm3: 0.23466479625314993

STABLE

cnn1:  0.2686938000036717
cnn2:  0.2672315995209581
cnn3:  0.2842348235240765
lstm1: 0.2866351047569809
lstm2: 0.2865176773619274
lstm3: 0.28638524453048186

OPEN: 

cnn1:  0.24325999158246783
cnn2:  0.23574411228558845
cnn3:  0.24901063811607702
lstm1: 0.2758464476557353
lstm2: 0.27620934328313484
lstm3: 0.27649455033626213

AGREEABLE:

cnn1:  0.19355836347180352
cnn2:  0.22527534064781957
cnn3:  0.21643586639481258
lstm1: 0.20851660443425266
lstm2: 0.20821464797410172
lstm3: 0.20789164186216755

CONSCIENTIOUS:

cnn1:  0.19135943288621196
cnn2:  0.20505606298317522
cnn3:  0.21620428965986255
lstm1: 0.2046760013249673
lstm2: 0.20467790279067985
lstm3: 0.20462718169507244


============================
============================
CLASSIFICATION:

EXTROVERTED <-0.3, 0.2)=0 <0.2, 0.5>=1:

cnn1:  0.6127
cnn2:  0.5352
cnn3:  0.6408
lstm1: 0.6408
lstm2: 0.5704
lstm3: 0.5775

============================
STABLE <-0.3, 0.2)=0 <0.2, 0.5>=1:

cnn1:  0.6056
cnn2:  0.6408
cnn3:  0.6972
lstm1: 0.6620
lstm2: 0.6197
lstm3: 0.6690

============================
OPEN:

cnn1:  0.6408
cnn2:  0.6268
cnn3:  0.6901
lstm1: 0.6408
lstm2: 0.6197
lstm3: 0.6479

============================
AGREE <-0.3, 0.2)=0 <0.2, 0.5>=1:

cnn1:  0.6268
cnn2:  0.5775
cnn3:  0.6549
lstm1: 0.6479
lstm2: 0.5704
lstm3: 0.6268

============================
CONSCIENTIOUS <-0.2, 0.2)=0 <0.2, 0.5>=1:

cnn1:  0.6056
cnn2:  0.6408
cnn3:  0.6127
lstm1: 0.6197
lstm2: 0.6127
lstm3: 0.6268
"""


'\ncnn1: word2vec embdedding + not trainable during fitting\ncnn2: default embdedding\ncnn2: word2vec embdedding + trainable during fitting\n============================\nEXTROVERTED <-0.3, 0.2)=0 <0.2, 0.5>=1:\n\ncnn1:  0.6127\ncnn2:  0.5352\ncnn3:  0.6408\nlstm1: 0.6408\nlstm2: 0.5704\nlstm3: 0.5775\n\n[0.20138032381716772,\n 0.19798711781557132,\n 0.1940063193601567,\n 0.23499138675457792,\n 0.23476451093520545,\n 0.23466479625314993]\n\nlonger embedding training:\ncnn1:  0.6127\ncnn2:  0.6056\ncnn3:  0.5915\nlstm1: 0.6127\nlstm2: 0.5845\nlstm2: 0.6268\n============================\nSTABLE <-0.3, 0.2)=0 <0.2, 0.5>=1:\n\ncnn1:  0.6056\ncnn2:  0.6408\ncnn3:  0.6972\nlstm1: 0.6620\nlstm2: 0.6197\nlstm3: 0.6690\n\n[0.2686938000036717,\n 0.2672315995209581,\n 0.2842348235240765,\n 0.2866351047569809,\n 0.2865176773619274,\n 0.28638524453048186]\n\nlonger embedding training:\ncnn1:  0.6761\ncnn2:  0.6408\ncnn3:  0.6338\nlstm1: 0.6620\nlstm2: 0.6761\nlstm3: 0.7042\n========================